# Эксперименты с Tesseract 5 по распознаванию удостоверений на тестовой выборке

## Оценка работы на тестовом датасете

In [ ]:
# Устанавливаем библиотеки
!sudo apt remove -y tesseract-ocr
!sudo apt autoremove -y
!sudo apt update
!sudo apt install -y software-properties-common
!sudo add-apt-repository -y ppa:alex-p/tesseract-ocr-devel  # Репозиторий с Tesseract 5
!sudo apt update
!sudo apt install -y tesseract-ocr libtesseract-dev
!sudo apt install -y tesseract-ocr-rus tesseract-ocr-eng  # Основные языки
!pip install pytesseract
!pip install python-Levenshtein tqdm opencv-python-headless -q

# Импорт библиотек
import os
import cv2
import json
import pytesseract
import pandas as pd
import numpy as np
from Levenshtein import distance as levenshtein_distance
from openpyxl import Workbook
from openpyxl.drawing.image import Image as ExcelImage
from PIL import Image as PILImage
import io

# Установка пути к данным Tesseract
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/5/tessdata/'

def preprocess_image(image):
    """
    Предобработка изображения: конвертация в оттенки серого.

    Args:
        image (numpy.ndarray): Входное изображение в формате BGR или grayscale.

    Returns:
        numpy.ndarray: Изображение в оттенках серого.
    """
    #Конвертация в grayscale
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    return gray

# Настройки путей
IMAGES_DIR = '/content/drive/MyDrive/VKR/kpk_dataset-9/test/images'
LABELS_DIR = '/content/drive/MyDrive/VKR/kpk_dataset-9/test/labels'
JSON_PATH = '/content/drive/MyDrive/VKR/kpk_dataset-9/test_annotations.json'
EXCEL_PATH = '/content/drive/MyDrive/VKR/results/results_optimized.xlsx'
DEBUG_DIR = '/content/drive/MyDrive/VKR/results/debug_images'
TEMP_IMG_DIR = '/content/drive/MyDrive/VKR/results/temp_images'
os.makedirs(DEBUG_DIR, exist_ok=True)
os.makedirs(TEMP_IMG_DIR, exist_ok=True)

# Сопоставление классов с их названиями
CLASS_MAPPING = {
    0: "city",
    1: "course_period",
    2: "course_topic",
    3: "hours",
    4: "name",
    5: "organization",
    6: "registration_number",
    7: "year"
}

def load_annotations(json_path):
    """
    Загрузка аннотаций из JSON-файла.

    Args:
        json_path (str): Путь к JSON-файлу с аннотациями.

    Returns:
        dict: Словарь с аннотациями для каждого изображения.
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return {os.path.splitext(item['file_name'])[0]: item['annotations'] for item in data['images']}

def read_yolo_bboxes(label_path, img_width, img_height):
    """
    Чтение bounding box из файла в формате YOLO.

    Args:
        label_path (str): Путь к файлу с разметкой.
        img_width (int): Ширина изображения.
        img_height (int): Высота изображения.

    Returns:
        list: Список bounding box в формате (class_id, (x1, y1, x2, y2)).
    """
    if not os.path.exists(label_path):
        return []

    bboxes = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue

            class_id, x_center, y_center, width, height = map(float, parts)
            x1 = int((x_center - width/2) * img_width)
            y1 = int((y_center - height/2) * img_height)
            x2 = int((x_center + width/2) * img_width)
            y2 = int((y_center + height/2) * img_height)
            bboxes.append((int(class_id), (x1, y1, x2, y2)))
    return bboxes

def clean_text(text):
    """
    Очистка текста: удаление лишних пробелов и переносов строк.

    Args:
        text (str): Входной текст.

    Returns:
        str: Очищенный текст.
    """
    return ' '.join(text.replace('\n', ' ').replace('\r', ' ').split()).strip()


def save_image_to_temp(image, filename):
   """
    Сохранение изображения во временную директорию.

    Args:
        image (numpy.ndarray): Изображение для сохранения.
        filename (str): Имя файла.

    Returns:
        str: Путь к сохраненному файлу.
    """
    temp_path = os.path.join(TEMP_IMG_DIR, filename)
    cv2.imwrite(temp_path, image)
    return temp_path

def process_image_with_stats(image_path, label_path, annotations, debug=False):
    """
    Обработка изображения: распознавание текста и расчет метрик.

    Args:
        image_path (str): Путь к изображению.
        label_path (str): Путь к файлу с разметкой.
        annotations (dict): Аннотации для изображений.
        debug (bool): Флаг отладки.

    Returns:
        list: Список результатов распознавания.
    """
    image = cv2.imread(image_path)
    if image is None:
        return []

    image_id = os.path.splitext(os.path.basename(image_path))[0]
    img_height, img_width = image.shape[:2]
    bboxes = read_yolo_bboxes(label_path, img_width, img_height)

    results = []
    for i, (class_id, bbox) in enumerate(bboxes):
        field_name = CLASS_MAPPING.get(class_id, f"Class {class_id}")
        cropped = image[bbox[1]:bbox[3], bbox[0]:bbox[2]]

        # Сохраняем изображение до обработки
        original_img_path = save_image_to_temp(cropped, f"{image_id}_{i}_original.png")

        # Обработка изображения
        processed = preprocess_image(cropped)

        # Сохраняем изображение после обработки
        processed_img_path = save_image_to_temp(processed, f"{image_id}_{i}_processed.png")

        # Распознавание
        text = pytesseract.image_to_string(processed, lang='rus+eng')
        recognized_text = clean_text(text)
        reference_text = clean_text(annotations.get(image_id, {}).get(field_name, ""))
        cer = calculate_cer(reference_text, recognized_text)

        results.append({
            "image_id": image_id,
            "class": field_name,
            "reference": reference_text,
            "recognized": recognized_text,
            "cer": cer,
            "original_image": original_img_path,
            "processed_image": processed_img_path
        })

    return results

def calculate_cer(reference, hypothesis):
    """
    Расчет Character Error Rate (CER).

    Args:
        reference (str): Эталонный текст.
        hypothesis (str): Распознанный текст.

    Returns:
        float: Значение CER.
    """
    if not reference:
        return 0.0 if not hypothesis else 1.0
    return levenshtein_distance(reference, hypothesis) / max(len(reference), 1)

def main():
    annotations = load_annotations(JSON_PATH)
    image_files = [f for f in os.listdir(IMAGES_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    all_results = []
    for i, image_file in enumerate(image_files[:15]):  # Ограничиваем для теста
        image_path = os.path.join(IMAGES_DIR, image_file)
        label_path = os.path.join(LABELS_DIR, os.path.splitext(image_file)[0] + ".txt")

        print(f"Обработка {i+1}/{len(image_files)}: {image_file}")
        results = process_image_with_stats(
            image_path, label_path, annotations,
            debug=(i < 3)  # Отладка для первых 3 изображений
        )
        all_results.extend(results)

    # Создаем DataFrame
    df = pd.DataFrame(all_results)

    # Создаем Excel файл с изображениями
    wb = Workbook()
    ws = wb.active
    ws.title = "Results"

    # Заголовки
    headers = ["Image ID", "Class", "Reference Text", "Recognized Text", "CER", "Original Image", "Processed Image"]
    ws.append(headers)

    # Добавляем данные и изображения
    for idx, row in df.iterrows():
        ws.append([
            row["image_id"],
            row["class"],
            row["reference"],
            row["recognized"],
            row["cer"],
            "",  # Место для изображения
            ""   # Место для изображения
        ])

        # Добавляем оригинальное изображение
        if os.path.exists(row["original_image"]):
            img = ExcelImage(row["original_image"])
            img.width = 100
            img.height = 100
            ws.add_image(img, f'F{idx + 2}')

        # Добавляем обработанное изображение
        if os.path.exists(row["processed_image"]):
            img = ExcelImage(row["processed_image"])
            img.width = 100
            img.height = 100
            ws.add_image(img, f'G{idx + 2}')

    # Сохраняем Excel файл
    wb.save(EXCEL_PATH)

    # Анализ результатов
    print("\nОбщая статистика CER:")
    print(f"Средний: {df['cer'].mean():.4f}")
    print(f"Медиана: {df['cer'].median():.4f}")

    print("\nПо классам:")
    print(df.groupby('class')['cer'].mean().sort_values())

    print(f"\nРезультаты сохранены в {EXCEL_PATH}")

if __name__ == "__main__":
    main()

## Оценка времени

In [ ]:
# Импорт библиотек
import time
import cv2
import os
import pandas as pd
from tqdm import tqdm
import json
import pytesseract
from Levenshtein import distance as levenshtein_distance

# Конфигурация путей
IMAGES_DIR = '/content/drive/MyDrive/VKR/kpk_dataset-9/test/images'
LABELS_DIR = '/content/drive/MyDrive/VKR/kpk_dataset-9/test/labels'
JSON_PATH = '/content/drive/MyDrive/VKR/kpk_dataset-9/test_annotations.json'
RESULTS_DIR = '/content/drive/MyDrive/VKR/results/tesseract'
os.makedirs(RESULTS_DIR, exist_ok=True)

CLASS_MAPPING = {
    0: "city",
    1: "course_period",
    2: "course_topic",
    3: "hours",
    4: "name",
    5: "organization",
    6: "registration_number",
    7: "year"
}

def load_annotations(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return {os.path.splitext(item['file_name'])[0]: item['annotations'] for item in data['images']}

def read_yolo_bboxes(label_path, img_width, img_height):
    if not os.path.exists(label_path):
        return []
    bboxes = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            class_id, x_center, y_center, width, height = map(float, parts)
            x1 = int((x_center - width/2) * img_width)
            y1 = int((y_center - height/2) * img_height)
            x2 = int((x_center + width/2) * img_width)
            y2 = int((y_center + height/2) * img_height)
            bboxes.append((int(class_id), (x1, y1, x2, y2)))
    return bboxes

def process_image_tesseract(image_path, label_path, annotations):
    image = cv2.imread(image_path)
    if image is None:
        return []

    image_id = os.path.splitext(os.path.basename(image_path))[0]
    img_height, img_width = image.shape[:2]
    bboxes = read_yolo_bboxes(label_path, img_width, img_height)

    results = []
    for class_id, bbox in bboxes:
        field_name = CLASS_MAPPING.get(class_id, f"Class {class_id}")
        cropped = image[bbox[1]:bbox[3], bbox[0]:bbox[2]]

        # Предобработка
        gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

        # Замер времени
        start_time = time.time()
        recognized_text = pytesseract.image_to_string(cropped, lang='rus+eng')
        elapsed = time.time() - start_time

        recognized_text = recognized_text.strip()
        reference_text = annotations.get(image_id, {}).get(field_name, "")

        results.append({
            'image_id': image_id,
            'class': field_name,
            'time': elapsed,
            'cer': levenshtein_distance(reference_text, recognized_text) / max(len(reference_text), 1)
        })

    return results

def main_tesseract():
    """
    Функция для оценки времени работы Tesseract.
    """
    annotations = load_annotations(JSON_PATH)
    image_files = [f for f in os.listdir(IMAGES_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    print(f"Начато тестирование Tesseract на {len(image_files)} изображениях")

    all_results = []
    for img_file in tqdm(image_files):
        image_path = os.path.join(IMAGES_DIR, img_file)
        label_path = os.path.join(LABELS_DIR, os.path.splitext(img_file)[0] + ".txt")

        results = process_image_tesseract(image_path, label_path, annotations)
        all_results.extend(results)

    # Сохранение и анализ результатов
    df = pd.DataFrame(all_results)
    df.to_csv(os.path.join(RESULTS_DIR, 'tesseract_results.csv'), index=False)

    print("\nРезультаты Tesseract:")
    print(f"Общее время: {df['time'].sum():.2f} сек")
    print(f"Среднее время: {df['time'].mean():.4f} сек/изображение")
    print(f"Средний CER: {df['cer'].mean():.4f}")

if __name__ == "__main__":
    main_tesseract()


## Интервальная оценка CER для Tesseract

In [ ]:
# Импорт библиотек
import numpy as np
from scipy import stats

RESULTS_DIR = '/content/drive/MyDrive/VKR/results/tesseract'

def calculate_confidence_interval_tesseract():
    """
    Функция для расчета доверительного интервала CER.
    """
    # Загрузка результатов
    df = pd.read_csv(os.path.join(RESULTS_DIR, 'tesseract_results.csv'))

    # Удаление нулевых строк (если есть проблемы с данными)
    df = df[df['cer'].notna()]

    # Расчет параметров
    mean_cer = df['cer'].mean()
    std_cer = df['cer'].std()
    n = len(df)

    # 95% доверительный интервал
    confidence = 0.95
    se = std_cer / np.sqrt(n)
    ci = stats.t.interval(confidence, n-1, loc=mean_cer, scale=se)

    print("\nИнтервальная оценка CER для Tesseract:")
    print(f"Средний CER: {mean_cer:.4f}")
    print(f"Стандартное отклонение: {std_cer:.4f}")
    print(f"95% доверительный интервал: ({ci[0]:.4f}, {ci[1]:.4f})")
    print(f"Количество образцов: {n}")

if __name__ == "__main__":
    calculate_confidence_interval_tesseract()